In [1]:
%cd ../..

/workspace


In [2]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import statsmodels.api as sm
import warnings
import random
from IPython.display import display, HTML
np.random.seed(42)
random.seed(42)
tqdm.pandas()

# Reading and Selecting Households

In [3]:
try:
    lclid_acorn_map = pd.read_pickle("data/london_smart_meters/preprocessed/london_smart_meters_lclid_acorn_map.pkl")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 02 - Preprocessing London Smart Meter Dataset.ipynb in Chapter02
    </div>
    """))

In [4]:
affluent_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Affluent", ["LCLid",'file']]
adversity_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Adversity", ["LCLid",'file']]
comfortable_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Comfortable", ["LCLid",'file']]


Let's take a subset of the data because if we take everything, it will hit your RAM. Depending on how much RAM you have, we can choose larger data. But to maintain the variety in the dataset, we will do stratified sampling based on Acorn classifications.

* <= 50 households for 4GB RAM
* 50 - 100 households for 8GB RAM
* 100-150 households for 16GB RAM
* 250 households for 32GB RAM

Let's sample 150 households now, but feel free to reduce of increase as per your hardware constraints

150 households means 50 each from the three Acorn Groups - Affluent, Comfortable, Adversity (we are ignoring the households with unknown ACORN groups)

In [5]:
selected_households = pd.concat(
    [
        affluent_households.sample(50, random_state=76),
        comfortable_households.sample(50, random_state=76),
        adversity_households.sample(50, random_state=76),
    ]
)
selected_households['block']=selected_households.file.str.split("_", expand=True).iloc[:,1].astype(int)

In [6]:
# extracting the paths to the different blocks and extracting the starting and ending blocks
path_blocks = [
    (p, *list(map(int, p.name.split("_")[5].split(".")[0].split("-"))))
    for p in Path("data/london_smart_meters/preprocessed").glob(
        "london_smart_meters_merged_block*"
    )
]

In [7]:
household_df_l = []
for path, start_b, end_b in tqdm(path_blocks):
    block_df = pd.read_parquet(path)
    selected_households['block'].between
    mask = selected_households['block'].between(start_b, end_b)
    lclids = selected_households.loc[mask, "LCLid"]
    household_df_l.append(block_df.loc[block_df.LCLid.isin(lclids)])

  0%|          | 0/14 [00:00<?, ?it/s]

In [8]:
block_df = pd.concat(household_df_l)
del household_df_l
block_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
4520,MAC001536,2012-05-23,30min,"[0.129, 0.124, 0.124, 0.046, 0.045, 0.045, 0.1...",31008,ToU,ACORN-L,Adversity,block_80,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[335, 335, 333, 333, 326, 326, 318, 318, 313, ...","[16.41, 16.41, 15.76, 15.76, 14.76, 14.76, 14....","[12.25, 12.25, 12.47, 12.47, 12.4, 12.4, 11.81...","[1019.95, 1019.95, 1020.32, 1020.32, 1020.6, 1...","[16.41, 16.41, 15.76, 15.76, 14.76, 14.76, 14....","[2.06, 2.06, 2.13, 2.13, 2.01, 2.01, 2.29, 2.2...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.76, 0.76, 0.81, 0.81, 0.86, 0.86, 0.85, 0.8...","[Clear, Clear, Clear, Clear, Clear, Clear, Cle..."
4583,MAC003416,2012-10-02,30min,"[0.073, 0.079, 0.052, 0.074, 0.081, 0.05599999...",24672,Std,ACORN-L,Adversity,block_81,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[218, 218, 219, 219, 218, 218, 215, 215, 214, ...","[11.78, 11.78, 11.37, 11.37, 10.89, 10.89, 10....","[10.3, 10.3, 10.17, 10.17, 9.65, 9.65, 9.62, 9...","[1012.35, 1012.35, 1012.4, 1012.4, 1012.2, 101...","[11.78, 11.78, 11.37, 11.37, 10.89, 10.89, 10....","[3.6, 3.6, 3.84, 3.84, 3.05, 3.05, 3.02, 3.02,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.91, 0.91, 0.92, 0.92, 0.92, 0.92, 0.92, 0.9...","[Clear, Clear, Clear, Clear, Clear, Clear, Cle..."
4594,MAC003836,2012-10-24,30min,"[0.186, 0.22, 0.24, 0.239, 0.291, 0.281, 0.278...",23616,Std,ACORN-L,Adversity,block_81,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[23, 23, 24, 24, 29, 29, 32, 32, 37, 37, 31, 3...","[13.13, 13.13, 13.57, 13.57, 13.54, 13.54, 13....","[13.06, 13.06, 13.03, 13.03, 12.67, 12.67, 12....","[1022.51, 1022.51, 1022.19, 1022.19, 1021.61, ...","[13.13, 13.13, 13.57, 13.57, 13.54, 13.54, 13....","[2.47, 2.47, 2.56, 2.56, 2.59, 2.59, 2.47, 2.4...","[rain, rain, rain, rain, rain, rain, rain, rai...","[fog, fog, fog, fog, fog, fog, fog, fog, fog, ...","[0.99, 0.99, 0.97, 0.97, 0.94, 0.94, 0.95, 0.9...","[Foggy, Foggy, Foggy, Foggy, Foggy, Foggy, Fog..."
4642,MAC003641,2012-10-13,30min,"[0.075, 0.0819999999999999, 0.042, 0.043, 0.04...",24144,Std,ACORN-L,Adversity,block_82,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[186, 186, 188, 188, 190, 190, 203, 203, 206, ...","[8.78, 8.78, 8.27, 8.27, 7.87, 7.87, 7.89, 7.8...","[6.28, 6.28, 6.21, 6.21, 6.22, 6.22, 6.76, 6.7...","[1007.7, 1007.7, 1007.36, 1007.36, 1006.73, 10...","[7.55, 7.55, 7.34, 7.34, 6.75, 6.75, 6.89, 6.8...","[2.28, 2.28, 1.81, 1.81, 1.95, 1.95, 1.83, 1.8...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.84, 0.84, 0.87, 0.87, 0.89, 0.89, 0.93, 0.9...","[Clear, Clear, Clear, Clear, Partly Cloudy, Pa..."
4660,MAC005529,2012-04-18,30min,"[0.242, 0.257, 0.237, 0.262, 0.239, 0.252, 0.2...",32688,ToU,ACORN-L,Adversity,block_82,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[197, 197, 195, 195, 183, 183, 173, 173, 173, ...","[6.31, 6.31, 6.35, 6.35, 6.38, 6.38, 6.44, 6.4...","[3.52, 3.52, 3.62, 3.62, 4.19, 4.19, 4.17, 4.1...","[992.26, 992.26, 991.04, 991.04, 989.79, 989.7...","[3.28, 3.28, 3.14, 3.14, 3.59, 3.59, 3.27, 3.2...","[4.35, 4.35, 4.75, 4.75, 3.92, 3.92, 4.72, 4.7...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.82, 0.82, 0.83, 0.83, 0.86, 0.86, 0.85, 0.8...","[Partly Cloudy, Partly Cloudy, Partly Cloudy, ..."


In [9]:
from src.utils.data_utils import compact_to_expanded

In [10]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/150 [00:00<?, ?it/s]

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-05-23 00:00:00,MAC001536,0.129,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,335,16.41,12.25,1019.95,16.41,2.06,rain,clear-night,0.76,Clear
1,2012-05-23 00:30:00,MAC001536,0.124,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,335,16.41,12.25,1019.95,16.41,2.06,rain,clear-night,0.76,Clear
2,2012-05-23 01:00:00,MAC001536,0.124,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,333,15.76,12.47,1020.32,15.76,2.13,rain,clear-night,0.81,Clear
3,2012-05-23 01:30:00,MAC001536,0.046,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,333,15.76,12.47,1020.32,15.76,2.13,rain,clear-night,0.81,Clear
4,2012-05-23 02:00:00,MAC001536,0.045,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,326,14.76,12.40,1020.60,14.76,2.01,rain,clear-night,0.86,Clear


## Reduce Memory Footprint

In [11]:
from src.utils.data_utils import reduce_memory_footprint

In [12]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4711440 entries, 0 to 32639
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 3.3 GB


In [13]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [14]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4711440 entries, 0 to 32639
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 301.1 MB


# Train Test Valildation Split

We are going to keep 2014 data as the validation and test period. We have 2 months(Jan and Feb) of data in 2014. Jan is Validation and Feb is Test

In [15]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 4293840 | # of Validation samples: 223200 | # of Test samples: 194400
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [16]:
train.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_train.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_val.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_test.parquet")

## Train Test Split after filling in missing values

In [17]:
from src.imputation.interpolation import SeasonalInterpolation

block_df.energy_consumption = block_df.energy_consumption.progress_apply(lambda x: SeasonalInterpolation(seasonal_period=48*7).fit_transform(x.reshape(-1,1)).squeeze())

  0%|          | 0/150 [00:00<?, ?it/s]

In [18]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/150 [00:00<?, ?it/s]

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-05-23 00:00:00,MAC001536,0.129,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,335,16.41,12.25,1019.95,16.41,2.06,rain,clear-night,0.76,Clear
1,2012-05-23 00:30:00,MAC001536,0.124,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,335,16.41,12.25,1019.95,16.41,2.06,rain,clear-night,0.76,Clear
2,2012-05-23 01:00:00,MAC001536,0.124,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,333,15.76,12.47,1020.32,15.76,2.13,rain,clear-night,0.81,Clear
3,2012-05-23 01:30:00,MAC001536,0.046,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,333,15.76,12.47,1020.32,15.76,2.13,rain,clear-night,0.81,Clear
4,2012-05-23 02:00:00,MAC001536,0.045,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,326,14.76,12.40,1020.60,14.76,2.01,rain,clear-night,0.86,Clear


## Reduce Memory Footprint

In [19]:
from src.utils.data_utils import reduce_memory_footprint

In [20]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4711440 entries, 0 to 32639
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 3.3 GB


In [21]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [22]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4711440 entries, 0 to 32639
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 301.1 MB


In [23]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 4293840 | # of Validation samples: 223200 | # of Test samples: 194400
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [24]:
train.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_train_missing_imputed.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_val_missing_imputed.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_test_missing_imputed.parquet")

In [28]:
train.head()

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-05-23 00:00:00,MAC001536,0.129,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,335,16.41,12.25,1019.950012,16.41,2.06,rain,clear-night,0.76,Clear
1,2012-05-23 00:30:00,MAC001536,0.124,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,335,16.41,12.25,1019.950012,16.41,2.06,rain,clear-night,0.76,Clear
2,2012-05-23 01:00:00,MAC001536,0.124,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,333,15.76,12.47,1020.320007,15.76,2.13,rain,clear-night,0.81,Clear
3,2012-05-23 01:30:00,MAC001536,0.046,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,333,15.76,12.47,1020.320007,15.76,2.13,rain,clear-night,0.81,Clear
4,2012-05-23 02:00:00,MAC001536,0.045,30min,31008,ToU,ACORN-L,Adversity,block_80,NO_HOLIDAY,...,326,14.76,12.40,1020.599976,14.76,2.01,rain,clear-night,0.86,Clear
